In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,10163.55,-0.03,0.01,-0.02,-0.01,-0.00,-0.0,0.00
3,-0.03,2097.13,-0.00,0.01,0.00,0.00,-0.0,-0.00
4,0.01,-0.00,379.05,-0.01,-0.00,0.00,-0.0,0.00
5,-0.02,0.01,-0.01,61.95,0.00,0.00,0.0,-0.00
6,-0.01,0.00,-0.00,0.00,9.72,-0.00,0.0,0.00
7,-0.00,0.00,0.00,0.00,-0.00,1.55,-0.0,-0.00
8,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.3,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00002,-0.00003,-0.00001,-0.00002,0.00000
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,-0.00000,-0.00001
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00004,-0.00001,0.00001
5,-0.00002,0.00002,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,-0.00000,-0.00001,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00000,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.05180540616124198

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.29243291e+07 5.38557453e+05 1.71070975e+04 4.38888158e+02
 1.01302234e+01 2.31431625e-01 6.72828093e-03 5.61556416e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999513,-0.030740,-0.005293,-0.000866,-0.000136,-0.000022,-0.000004,-6.403471e-07
1,0.030555,0.999031,-0.031273,-0.004917,-0.000771,-0.000123,-0.000024,-3.767682e-06
2,0.006211,0.030910,0.998920,-0.033722,-0.005124,-0.000815,-0.000161,-2.449302e-05
3,0.001217,0.005889,0.033310,0.998594,-0.040287,-0.006248,-0.001225,-1.884523e-04
4,0.000238,0.001150,0.006373,0.039674,0.997637,-0.054719,-0.010382,-1.588458e-03
5,0.000050,0.000241,0.001335,0.008183,0.053071,0.993311,-0.101160,-1.464441e-02
6,0.000015,0.000070,0.000390,0.002384,0.015211,0.097031,0.983211,-1.537658e-01
7,0.000005,0.000022,0.000122,0.000746,0.004750,0.029735,0.151504,9.879975e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004869848939657029,
 0.0009686794731782467,
 0.0010796418975020927,
 0.001405646967519436,
 0.002363002290778682,
 0.006688903542800495,
 0.016789024749444104,
 0.01200249572830725]